In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import srsly

C:\Users\ehug0006\Miniconda3\envs\autumn310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ehug0006\Miniconda3\envs\autumn310\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
C:\Users\ehug0006\Miniconda3\envs\autumn310\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [40]:
# Data from WHO covid dashboard: https://data.who.int/dashboards/covid19/data

case_df = pd.read_csv(r'C:\Users\ehug0006\Downloads\WHO-COVID-19-global-data.csv')

In [5]:
case_df.head()

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-01-05,AF,Afghanistan,EMRO,NaN,0,NaN,0
1,2020-01-12,AF,Afghanistan,EMRO,NaN,0,NaN,0
2,2020-01-19,AF,Afghanistan,EMRO,NaN,0,NaN,0
3,2020-01-26,AF,Afghanistan,EMRO,NaN,0,NaN,0
4,2020-02-02,AF,Afghanistan,EMRO,NaN,0,NaN,0


In [107]:
case_df

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-01-05,AF,Afghanistan,EMRO,NaN,0,NaN,0
1,2020-01-12,AF,Afghanistan,EMRO,NaN,0,NaN,0
2,2020-01-19,AF,Afghanistan,EMRO,NaN,0,NaN,0
3,2020-01-26,AF,Afghanistan,EMRO,NaN,0,NaN,0
4,2020-02-02,AF,Afghanistan,EMRO,NaN,0,NaN,0
...,...,...,...,...,...,...,...,...
53755,2024-03-17,ZW,Zimbabwe,AFRO,NaN,266359,NaN,5740
53756,2024-03-24,ZW,Zimbabwe,AFRO,NaN,266359,NaN,5740
53757,2024-03-31,ZW,Zimbabwe,AFRO,NaN,266359,NaN,5740
53758,2024-04-07,ZW,Zimbabwe,AFRO,NaN,266359,NaN,5740


In [13]:
# select countries for analysis

countries = ['Malaysia', 'Philippines', 'Viet Nam']

analysis_df = case_df.loc[case_df['Country'].isin(countries)]

In [20]:
fig = px.scatter(analysis_df, 
              x='Date_reported', y='New_cases', 
              color='Country',
             log_y=False)

fig.show()

In [ ]:
# Code for grabbing variant props for covariants github
# Data from covariants cluster tables https://github.com/hodcroftlab/covariants/tree/master/cluster_tables, covariants.org

In [2]:
# Read json files into dictionaries
Beta_20H = srsly.read_json(r'C:\Users\ehug0006\emu\variant_data_covariants\20H.Beta.V2_data.json')
Alpha_20I = srsly.read_json(r'C:\Users\ehug0006\emu\variant_data_covariants\20I.Alpha.V1_data.json')
Delta_21A = srsly.read_json(r'C:\Users\ehug0006\emu\variant_data_covariants\21A.Delta_data.json')
Delta_21I = srsly.read_json(r'C:\Users\ehug0006\emu\variant_data_covariants\21I.Delta_data.json')
Delta_21J = srsly.read_json(r'C:\Users\ehug0006\emu\variant_data_covariants\21J.Delta_data.json')
Omicron_21K_BA1 = srsly.read_json(r'C:\Users\ehug0006\emu\variant_data_covariants\21K.Omicron_data.json')
Omicron_21L_BA2 = srsly.read_json(r'C:\Users\ehug0006\emu\variant_data_covariants\21L.Omicron_data.json')
Omicron_22A_BA4 = srsly.read_json(r'C:\Users\ehug0006\emu\variant_data_covariants\22A.Omicron_data.json')
Omicron_22B_BA5 = srsly.read_json(r'C:\Users\ehug0006\emu\variant_data_covariants\22B.Omicron_data.json')

In [3]:
# Combine variant dictionaries into single nested dictionary
variant_data_dict = {'Beta_20H':Beta_20H,
                    'Alpha_20I':Alpha_20I,
                     'Delta_21A':Delta_21A,
                     'Delta_21I':Delta_21I,
                     'Delta_21J': Delta_21J,
                     'Omicron_21K_BA1': Omicron_21K_BA1,
                     'Omicron_21L_BA2': Omicron_21L_BA2,
                     'Omicron_22A_BA4': Omicron_22A_BA4,
                     'Omicron_22B_BA5':Omicron_22B_BA5
                    }

In [4]:
# create dataframe from nested variant dictionary
df = pd.DataFrame()

for variant,country in variant_data_dict.items():
    for c in country:
        new_df = pd.DataFrame(country[c])
        new_df['country'] = c
        new_df['variant'] = variant
        df = pd.concat([df, new_df])
        
# create new variant proportion column
df['variant_prop'] = df['cluster_sequences']/df['total_sequences']

In [6]:
# select countries
countries = ['Malaysia', 'Philippines', 'Vietnam']

df_analysis = df.loc[df['country'].isin(countries)]

In [19]:
# identify week first time proportion goes above a threshold
threshold = 0.01

emergence_df = df_analysis.loc[df_analysis.variant_prop>threshold].groupby(['country','variant']).head(2)

In [20]:
emergence_df

,week,total_sequences,cluster_sequences,country,variant,variant_prop
19,2021-02-01,376,19,Philippines,Beta_20H,0.050532
20,2021-02-15,327,74,Philippines,Beta_20H,0.226300
13,2020-11-23,41,1,Malaysia,Beta_20H,0.024390
18,2021-02-01,87,2,Malaysia,Beta_20H,0.022989
10,2020-12-21,3,2,Vietnam,Alpha_20I,0.666667
11,2021-01-04,2,2,Vietnam,Alpha_20I,1.000000
16,2020-12-21,201,4,Philippines,Alpha_20I,0.019900
17,2021-01-04,446,31,Philippines,Alpha_20I,0.069507
15,2020-12-21,36,1,Malaysia,Alpha_20I,0.027778
18,2021-02-01,87,1,Malaysia,Alpha_20I,0.011494
